# MSK Connection Test Project

## Purpose

The **MSK Connection Test Project** aims to assess the performance and connectivity of an **Amazon Managed Streaming for Apache Kafka (MSK)** cluster. The project provides a comprehensive testing framework to validate the following aspects of the MSK setup:

- **Cluster Connectivity:** Ensuring secure and reliable connectivity to the MSK cluster using IAM-based SASL authentication.
- **Producer Performance:** Measuring the ability to send messages to Kafka topics with proper error handling.
- **Consumer Performance:** Validating the consumption of messages from Kafka topics with accurate metrics.
- **Latency and Throughput:** Evaluating message delivery speed and capacity under varying load conditions.

## Scope

The project is designed for:

- Performance benchmarking of an **MSK** cluster.
- Security validation using **IAM-based SASL authentication**.
- End-to-end testing of Kafka producer and consumer components.

## Key Features

- **Kafka Producer and Consumer Implementation:** The project includes Python-based implementations for both producer and consumer using the `kafka-python` library.
- **IAM Authentication Integration:** Security is enforced through **MSK IAM SASL Signer** for authentication.
- **Automated Topic Management:** The project can create and manage Kafka topics programmatically.
- **Performance Metrics Collection:** Throughput, latency, and error rates are collected and visualized using Matplotlib.

## Technologies Used

- **Python 3.9.7**
- **Amazon MSK**
- **boto3** (AWS SDK for Python)
- **kafka-python**
- **aws-msk-iam-sasl-signer-python**

## Project Workflow

1. **Environment Setup:**
   - Install required libraries (`boto3`, `kafka-python`, `aws-msk-iam-sasl-signer-python`).
2. **Cluster Connection:**
   - Initialize the MSK IAM Token Provider.
   - Establish connection to the MSK cluster using public bootstrap servers.
3. **Topic Management:**
   - Programmatically create a new Kafka topic.
4. **Producer Operations:**
   - Send test messages to a Kafka topic and log the delivery status.
5. **Consumer Operations:**
   - Consume messages from the Kafka topic and log received messages.

## Usage Instructions

1. **Run the Notebook:** Execute the cells sequentially.
2. **Modify Cluster Details:** Update the `bootstrap_servers` and `region` variables as needed.
3. **Verify Message Delivery:** Check producer and consumer logs for successful message exchange.

## Conclusion

This project serves as a foundational tool for validating MSK cluster connectivity and performance. It can be extended for larger-scale tests and custom data generation scenarios.


In [1]:
!pip install kafka-python

     |████████████████████████████████| 246 kB 6.3 MB/s eta 0:00:01


In [2]:
!pip install aws-msk-iam-sasl-signer-python

     |████████████████████████████████| 13.3 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 39.8 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 2.1 MB/s  eta 0:00:01
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)


In [11]:
!pip install matplotlib

In [45]:
from kafka.errors import KafkaError
from kafka.admin import KafkaAdminClient, NewTopic
import socket
import time
from aws_msk_iam_sasl_signer import MSKAuthTokenProvider
import boto3
import json
import time
from kafka import KafkaProducer,KafkaConsumer


In [4]:
region = "us-east-1"

# MSKTokenProvider Class Documentation

## Purpose

The `MSKTokenProvider` class is designed to generate an authentication token for **Amazon Managed Streaming for Apache Kafka (MSK)** using **IAM-based SASL authentication**. This token is required for secure communication with the MSK cluster and ensures only authorized clients can interact with the cluster.


In [5]:
class MSKTokenProvider():
    def token(self):
        token, _ = MSKAuthTokenProvider.generate_auth_token(region)
        return token

tp = MSKTokenProvider()

# KafkaAdminClient Configuration Documentation

## Purpose

The code initializes a **KafkaAdminClient** for interacting with an **Amazon Managed Streaming for Apache Kafka (MSK)** cluster. This client is used to manage and administer Kafka resources such as topics, partitions, and configurations in a secure manner.



In [6]:
admin_client = KafkaAdminClient(
    bootstrap_servers="b-2-public.leonardocluster.ykfidp.c14.kafka.us-east-1.amazonaws.com:9198,b-1-public.leonardocluster.ykfidp.c14.kafka.us-east-1.amazonaws.com:9198",
    security_protocol="SASL_SSL",
    sasl_mechanism="OAUTHBEARER",
    sasl_oauth_token_provider=tp,
    client_id=socket.gethostname(),
)

In [7]:
def create_topic(topic_name, num_partitions, replication_factor):
    """Create a Kafka topic using MSK IAM authentication."""
    topic = NewTopic(name=topic_name, num_partitions=num_partitions, replication_factor=replication_factor)
    #response = admin_client.create_topics([topic])
    #print(response)
    try:
        response = admin_client.create_topics([topic])
        # Fix: Correctly accessing the tuple structure
        for topic_error in response.topic_errors:
            topic_name, error_code, error_message = topic_error
            if error_code == 0:
                print(f"Topic '{topic_name}' successfully created.")
            else:
                print(f"Error creating topic '{topic_name}': {error_message}")
    except KafkaError as e:
        print(f"KafkaError occurred: {e}")
    except Exception as e:
        print(f"Failed to create topic '{topic_name}': {e}")
    finally:
        print("Topic creation attempt complete.")

In [63]:
create_topic("pokemon-gen-6-topic",3,2)

Topic 'pokemon-gen-6-topic' successfully created.
Topic creation attempt complete.


In [30]:
bootstrap_servers = ["b-1-public.leonardocluster.ykfidp.c14.kafka.us-east-1.amazonaws.com:9198","b-2-public.leonardocluster.ykfidp.c14.kafka.us-east-1.amazonaws.com:9198"]
topic_name = "leonardo-msk-topic"

In [65]:
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    security_protocol="SASL_SSL",
    sasl_mechanism="OAUTHBEARER",
    sasl_oauth_token_provider=tp,
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    request_timeout_ms=240000,  # Increase the request timeout to 120 seconds
    client_id=socket.gethostname()
)

def send_message(topic_name, message):
    """Send a message to the specified Kafka topic."""
    try:
        future = producer.send(topic_name, value=message)
        record_metadata = future.get(timeout=10)
        print(f"Message sent to {record_metadata.topic} partition {record_metadata.partition} offset {record_metadata.offset}")
    except Exception as e:
        print(f"Failed to send message: {e}")

In [50]:
send_message(topic_name, {"event": "Hello Leonardo!", "type": "greeting"})
producer.close()

Message sent to leonardo-msk-topic partition 1 offset 1


In [53]:
import pandas as pd

In [54]:
pokemon_file = "pokemon.csv"
df_pokemon = pd.read_csv(pokemon_file,encoding='utf-8')

In [66]:
df_pokemon[0:10]

,id,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [56]:
df_pokemon.rename(columns={"#": "id"}, inplace=True)


In [67]:
import json

for index, row in df_pokemon.iterrows():
    poke_json = {
        "id": row['id'],
        "Name": row['Name'],
        "Type 1": row['Type 1'],
        "Type 2": row['Type 2'],
        "Total": row['Total'],
        "HP": row['HP'],
        "Attack": row['Attack'],
        "Defense": row['Defense'],
        "Sp. Atk": row['Sp. Atk'],
        "Sp. Def": row['Sp. Def'],
        "Speed": row['Speed'],
        "Generation": row['Generation'],
        "Legendary": row['Legendary']
    }
    send_message("pokemon-gen-{}-topic".format(row['Generation']), poke_json)
producer.close()


Message sent to pokemon-gen-1-topic partition 1 offset 0
Message sent to pokemon-gen-1-topic partition 1 offset 1
Message sent to pokemon-gen-1-topic partition 2 offset 0
Message sent to pokemon-gen-1-topic partition 0 offset 0
Message sent to pokemon-gen-1-topic partition 0 offset 1
Message sent to pokemon-gen-1-topic partition 1 offset 2
Message sent to pokemon-gen-1-topic partition 2 offset 1
Message sent to pokemon-gen-1-topic partition 2 offset 2
Message sent to pokemon-gen-1-topic partition 2 offset 3
Message sent to pokemon-gen-1-topic partition 1 offset 3
Message sent to pokemon-gen-1-topic partition 2 offset 4
Message sent to pokemon-gen-1-topic partition 1 offset 4
Message sent to pokemon-gen-1-topic partition 0 offset 2
Message sent to pokemon-gen-1-topic partition 1 offset 5
Message sent to pokemon-gen-1-topic partition 2 offset 5
Message sent to pokemon-gen-1-topic partition 2 offset 6
Message sent to pokemon-gen-1-topic partition 1 offset 6
Message sent to pokemon-gen-1-t

In [68]:
consumer = KafkaConsumer(
    "pokemon-gen-1-topic",
    bootstrap_servers=bootstrap_servers,
    security_protocol="SASL_SSL",
    sasl_mechanism="OAUTHBEARER",
    sasl_oauth_token_provider=tp,
    value_deserializer=lambda v: json.loads(v.decode('utf-8')),
    group_id="pokemon-gen-1-group",
    client_id=socket.gethostname(),
    auto_offset_reset="earliest"
)

# Consume messages from the Kafka topic
def consume_messages():
    try:
        print("Starting consumer...")
        for message in consumer:
            print(f"Received message: {message.value} from topic {message.topic} partition {message.partition} offset {message.offset}")
    except Exception as e:
        print(f"Error while consuming messages: {e}")
    finally:
        consumer.close()

In [ ]:
consume_messages()

Starting consumer...
Received message: {'id': 1, 'Name': 'Bulbasaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': 318, 'HP': 45, 'Attack': 49, 'Defense': 49, 'Sp. Atk': 65, 'Sp. Def': 65, 'Speed': 45, 'Generation': 1, 'Legendary': False} from topic pokemon-gen-1-topic partition 1 offset 0
Received message: {'id': 2, 'Name': 'Ivysaur', 'Type 1': 'Grass', 'Type 2': 'Poison', 'Total': 405, 'HP': 60, 'Attack': 62, 'Defense': 63, 'Sp. Atk': 80, 'Sp. Def': 80, 'Speed': 60, 'Generation': 1, 'Legendary': False} from topic pokemon-gen-1-topic partition 1 offset 1
Received message: {'id': 5, 'Name': 'Charmeleon', 'Type 1': 'Fire', 'Type 2': nan, 'Total': 405, 'HP': 58, 'Attack': 64, 'Defense': 58, 'Sp. Atk': 80, 'Sp. Def': 65, 'Speed': 80, 'Generation': 1, 'Legendary': False} from topic pokemon-gen-1-topic partition 1 offset 2
Received message: {'id': 7, 'Name': 'Squirtle', 'Type 1': 'Water', 'Type 2': nan, 'Total': 314, 'HP': 44, 'Attack': 48, 'Defense': 65, 'Sp. Atk': 50, 'Sp. Def': 64, 'S